# Predicting Spectrum Traits using Partial Least Squares Regression

In [1]:
# Import libraries necessary for this project
from torch.utils.data import Dataset, DataLoader
import os
import HyperSpectrumDataSet as hsd
import sys
import datetime
import copy
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.signal import savgol_filter 
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler 
from sklearn import linear_model 
from sklearn.model_selection import cross_val_predict 
from sklearn.metrics import mean_squared_error, r2_score
# Pretty display for notebooks
%matplotlib inline

In [2]:
from sys import stdout

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.signal import savgol_filter

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn import model_selection
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale 


In [3]:
def searchCV(X_train, y_train):
    
    # 10-fold CV, with shuffle
    kf_10 = model_selection.KFold(n_splits=10, shuffle=True, random_state=1)
    
    best_rsquare = -99999.00
    best_pc = 0
    for i in np.arange(1, 20):
        pls = PLSRegression(n_components=i)
        
        score_cv = model_selection.cross_val_score(pls, X_train, y_train, cv=kf_10, 
                                                   scoring='r2').mean()
        if best_rsquare < score_cv:
            best_rsquare = score_cv
            best_pc = i

    return best_pc


In [10]:

drop_percent = 0.2

exp_desc = "PLS_drop_"+str(drop_percent)

all_traits = ["LMA_O", "Narea_O", "SPAD_O", "Nmass_O", "Parea_O", "Pmass_O", "Vcmax", "Vcmax25", "J", "Photo_O", "Cond_O"]

run_traits = all_traits

train_results = {}
test_results = {}

train_sum = 0
test_sum = 0

for target_trait in run_traits:
    train_ds = hsd.HyperSpectrumDataSet(target_trait, "train", drop_percent=drop_percent)
    val_ds = hsd.HyperSpectrumDataSet(target_trait, "val")
    test_ds = hsd.HyperSpectrumDataSet(target_trait, "test")
        
    X_test, y_test = test_ds.data_values, test_ds.data_labels   
    X_train = train_ds.data_values.append(val_ds.data_values)
    y_train = train_ds.data_labels.append(val_ds.data_labels)
    
    y_train = y_train.values.ravel()
    y_test = y_test.values.ravel()

    best_pc = searchCV(X_train, y_train)
    
    pls = PLSRegression(n_components=best_pc)
    
    pls.fit(X_train, y_train)
    # Prediction
    train_y_pred = pls.predict(X_train)
    test_y_pred = pls.predict(X_test)
    
    # Calculate scores
    train_rsquare = r2_score(y_train, train_y_pred)
    test_rsquare = r2_score(y_test, test_y_pred)
    
    train_results[target_trait] = train_rsquare
    test_results[target_trait] = test_rsquare

    print(target_trait, train_rsquare, test_rsquare)


LMA_O 0.921016400848925 0.8696748778196837
Narea_O 0.9471510087509108 0.8827471226731548
SPAD_O 0.881399944299598 0.8294661494100277
Nmass_O 0.8458232536928861 0.7558889072383026
Parea_O 0.36449772901784616 0.4884083777093243
Pmass_O 0.6723436768450183 0.4662706782257545
Vcmax 0.8500419997097894 0.7211730677889432
Vcmax25 0.8253122648781408 0.48501256176588825
J 0.8472022953590705 0.8034708347126309
Photo_O 0.6654168754101966 0.6776440805428099
Cond_O 0.589858814072022 0.4537750879331717


In [11]:
# print report to file

now = datetime.datetime.now()

script_dir = os.path.dirname("__file__")

result_file_name = os.path.join(script_dir, "results")
result_file_name = os.path.join(result_file_name, exp_desc + "_" + now.strftime("%Y-%m-%d-%H-%M")+"_results.txt")

result_file = open(result_file_name,"w")
result_file.write("-------------------------------------------------\n")
result_file.write("Here are the training data set results:\n")

total = 0.0
for key, value in train_results.items():
    result_file.write("trait: " + str(key) +", R square: " + str(np.asscalar(value))+"\n")
    total += np.asscalar(value)
result_file.write("Average R square: " + str(total / len(train_results))+"\n")
result_file.write("-------------------------------------------------\n")

result_file.write("Here are the test data set results: \n")
total = 0.0

for key, value in test_results.items():
    result_file.write("trait: " + str(key) + ", R square: " + str(np.asscalar(value))+"\n")
    total += np.asscalar(value)
result_file.write("Average R square: " + str(total / len(test_results)) + "\n")
result_file.write("-------------------------------------------------\n")
result_file.close()

result_file = open(result_file_name+"csv","w")
result_file.write("Model, Dataset, LMA_O, Narea_O, SPAD_O, Nmass_O, Parea_O, Pmass_O, Vcmax, Vcmax25, J, Photo_O, Cond_O, Average\n")

result_file.write(exp_desc +", train, ") 
total = 0.0

for trait in all_traits:
    if trait in train_results:
        result_file.write(str(np.asscalar(train_results[trait]))+", ")
        total += np.asscalar(train_results[trait])
    else:
        result_file.write(", ")

result_file.write(str(total/len(train_results))+"\n")

result_file.write(exp_desc +", test, ") 
total = 0.0

for trait in all_traits:
    if trait in test_results:
        result_file.write(str(np.asscalar(test_results[trait]))+", ")
        total += np.asscalar(test_results[trait])
    else:
        result_file.write(", ")

result_file.write(str(total/len(test_results))+"\n")

result_file.close()